In [1]:
pip install tensorflow-datasets

In [2]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

# Load the dataset
dataset = tfds.load('tiny_shakespeare', as_supervised=False)
text_dataset = dataset['train']

# Extract text and convert to string
for example in text_dataset.take(1):
    text = example['text'].numpy().decode('utf-8')

# Create a mapping from unique characters to indices
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# Convert all text into integers
text_as_int = np.array([char2idx[c] for c in text])

# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

# Create training examples and targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/1.0.0.incompleteUA3HCG/tiny_shakespeare-train.tfrecord*..…

Generating validation examples...:   0%|          | 0/1 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/1.0.0.incompleteUA3HCG/tiny_shakespeare-validation.tfreco…

Generating test examples...:   0%|          | 0/1 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/1.0.0.incompleteUA3HCG/tiny_shakespeare-test.tfrecord*...…

Dataset tiny_shakespeare downloaded and prepared to /root/tensorflow_datasets/tiny_shakespeare/1.0.0. Subsequent calls will reuse this data.


In [3]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

# Model parameters
vocab_size = len(vocab)
embedding_dim = 512
rnn_units = 2048

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=64)


In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 512)           33280     
                                                                 
 lstm (LSTM)                 (64, None, 2048)          20979712  
                                                                 
 dense (Dense)               (64, None, 65)            133185    
                                                                 
Total params: 21146177 (80.67 MB)
Trainable params: 21146177 (80.67 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

# Train the model
EPOCHS = 50
history = model.fit(dataset, epochs=EPOCHS)


Epoch 1/50
155/155 [==============================] - 9s 27ms/step - loss: 2.6750
Epoch 2/50
155/155 [==============================] - 5s 25ms/step - loss: 1.9066
Epoch 3/50
 59/155 [==========>...................] - ETA: 2s - loss: 1.6961

In [ ]:
# Save the model weights
model.save_weights('./my_model_weights')

In [ ]:
def generate_text(model, start_string, temperature=1.0):
    # Number of characters to generate
    num_generate = 1000

    # Convert start_string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store results
    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        # Use a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))
# Recreate the model with batch size of 1 for inference
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.build(tf.TensorShape([1, None]))

# Load the previously saved weights
model.load_weights('./my_model_weights')

# Now the model is ready to generate text
print(generate_text(model, start_string="ROMEO: "))